# Genera los TFRecords en base a los XMLs correspondientes a las imágenes

Adaptado de https://github.com/douglasrizzo/detection_util_scripts 

Fuente: https://medium.com/@omcar17/how-to-convert-xml-files-into-tfrecords-in-tensorflow2-0-86120b553f0b


0) Preparar el ambiente:

In [1]:
#@title Actualizar e instalar paquetes necesarios
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim
!pip install pycocotools

     |████████████████████████████████| 352 kB 5.1 MB/s 


In [2]:
#@title Clonar el repositorio de modelos de TF si no está ya disponible
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2894, done.
remote: Counting objects: 100% (2894/2894), done.
remote: Compressing objects: 100% (2423/2423), done.
remote: Total 2894 (delta 733), reused 1297 (delta 433), pack-reused 0
Receiving objects: 100% (2894/2894), 32.92 MiB | 28.47 MiB/s, done.
Resolving deltas: 100% (733/733), done.


In [3]:
#@title Instalar el Object Detection API
# Nota: si falla por falta de requerimientos, ejecutarlo de nuevo y funcionará ;)...
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1665159 sha256=85ce5a6416116299bc870a6da877091afec9757cde1f1df67029346d587f3941
  Stored in directory: /tmp/pip-ephem-wheel-cache-8le89esw/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22258 sha256=f3ebab0e41e91f7ad994cf5ac1436368ccdab176a7c323d8b61c163a22b34e5e
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-py3-none-any.whl size=43512 sha256=06889ef7160724d22c857f3cf7fe58a3bbdfb1882b8904c309c3064b24fb3787
  Stored in directory: /root/.cache/pip/wheels/bc/49/5f/fdb5b9d85055c478213e0158ac122b596816149a02d82e0ab1
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78546 sha256=420762eeb4ead7aeb0c4d2aad9342b45cf242fd338c33d53c

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
#@title Re-Instalar el Object Detection API
# Nota: se ejecuta de nuevo para que lo instale bien
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1665159 sha256=3eb5702f5abaf52ff7eed26c3c5a067ff28eafc453905a8c134ca902354153f1
  Stored in directory: /tmp/pip-ephem-wheel-cache-_lqrnh33/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


1) Define las librerías a utilizar:

In [5]:
#@title Cargar Librerías
import os
import glob
import pandas as pd

import xml.etree.ElementTree as ET

import io
import tensorflow as tf

from PIL import Image
from tqdm import tqdm
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

print("Librerías cargadas.")

Librerías cargadas.


2) Monta el Drive y define archivos a utilizar:

In [6]:
#@title Montar Drive y definir archivos a procesar

# monta Google Drive nuevamente (se pierde conexión cuando se reinicia el entrorno anterior)
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# configuración de directorios local en Google Drive
drive_path = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas'
data_dir_path = drive_path + '/Cursogramas'

print("\n")
print("> Datos disponibles en: ", data_dir_path)
print("\n")

# define las carpetas de XMLs
train_image_dir = data_dir_path + '/train/images'
train_xml_dir = data_dir_path + '/train/annotations'
test_image_dir = data_dir_path + '/validation/images'
test_xml_dir = data_dir_path + '/validation/annotations' 
print("> train_image_dir: ", train_image_dir)
print("> train_xml_dir: ", train_xml_dir)
print("> test_image_dir: ", test_image_dir)
print("> test_xml_dir: ", test_xml_dir)
print("\n")

# define los nombres de los archivos
train_csv_fname = data_dir_path + '/train_labels.csv'
test_csv_fname = data_dir_path + '/test_labels.csv'
print("> train_csv_fname: ", train_csv_fname)
print("> test_csv_fname: ", test_csv_fname)

label_map_pbtxt_fname = data_dir_path + '/label_map.pbtxt'
print("> label_map_pbtxt_fname: ", label_map_pbtxt_fname)

train_record_fname = data_dir_path + '/train.record'
test_record_fname = data_dir_path + '/test.record'
print("> train_record_fname: ", train_record_fname)
print("> test_record_fname: ", test_record_fname)

Mounted at /content/gdrive


> Datos disponibles en:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas


> train_image_dir:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/train/images
> train_xml_dir:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/train/annotations
> test_image_dir:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/validation/images
> test_xml_dir:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/validation/annotations


> train_csv_fname:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/train_labels.csv
> test_csv_fname:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/test_labels.csv
> label_map_pbtxt_fname:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/label_map.pbtxt
> train_record_fname:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/train.record
> test_record_fname:  /content/gdrive/My Dr

3) Genera CSVs auxiliares en base a los XMLs correspondientes a las imágenes: 

In [7]:
#@title Definir funciones auxiliares para CSV
def __list_to_csv(annotations, output_file):
    column_name = [
        'filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'
    ]
    xml_df = pd.DataFrame(annotations, columns=column_name)
    xml_df.to_csv(output_file, index=None)

def xml_to_csv(xml_dir, output_file):
    """Reads all XML files, generated by labelImg, from a directory and generates a single CSV file"""
    annotations = []
    xmlFileList = glob.glob(xml_dir + '/*.xml')
    total = len(xmlFileList)
    auxMostrarTotal = (total//20)
    cant = 0
    print("XMLs a procesar en ", xml_dir, ": ", total)
    for xml_file in xmlFileList:
        cant = cant + 1
        if (cant % auxMostrarTotal) == 0:
          print("\t\t ya se procesaron  ", cant, " xmls")
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text), member[0].text,
                     int(member[4][0].text), int(member[4][1].text),
                     int(member[4][2].text), int(member[4][3].text))
            annotations.append(value)

    __list_to_csv(annotations, output_file)
    print(output_file, " generado. ")

print("Funciones auxiliares para CSV definidas")    

Funciones auxiliares para CSV definidas


In [8]:
#@title Generar CSVs

# Por las dudas, se elimina los archivos si ya existen (para asegurar su actualización)
if os.path.isfile(train_csv_fname):
  os.remove(train_csv_fname)
  print(train_csv_fname, " eliminado.")

if os.path.isfile(test_csv_fname):
  os.remove(test_csv_fname)
  print(test_csv_fname, " eliminado.")

# genera CSVs
print("\n")
print("-- generando train_csv: ", train_csv_fname)
xml_to_csv(train_xml_dir, train_csv_fname)
print("> train_csv generado.")
print("\n")
print("-- generando test_csv: ", test_csv_fname)
xml_to_csv(test_xml_dir, test_csv_fname)
print("> test_csv generado.")



-- generando train_csv:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/train_labels.csv
XMLs a procesar en  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/train/annotations :  3110
		 ya se procesaron   155  xmls
		 ya se procesaron   310  xmls
		 ya se procesaron   465  xmls
		 ya se procesaron   620  xmls
		 ya se procesaron   775  xmls
		 ya se procesaron   930  xmls
		 ya se procesaron   1085  xmls
		 ya se procesaron   1240  xmls
		 ya se procesaron   1395  xmls
		 ya se procesaron   1550  xmls
		 ya se procesaron   1705  xmls
		 ya se procesaron   1860  xmls
		 ya se procesaron   2015  xmls
		 ya se procesaron   2170  xmls
		 ya se procesaron   2325  xmls
		 ya se procesaron   2480  xmls
		 ya se procesaron   2635  xmls
		 ya se procesaron   2790  xmls
		 ya se procesaron   2945  xmls
		 ya se procesaron   3100  xmls
/content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/train_labels.csv  generado. 
> train_csv generado.


--

4) Genera archivo de clases label_map_pbtxt:

In [9]:
#@title Definir funciones auxiliares para label_map
def pbtxt_from_csv(csv_path, pbtxt_path):
    class_list = list(pd.read_csv(csv_path)['class'].unique())
    class_list.sort()

    pbtxt_from_classlist(class_list, pbtxt_path)


def pbtxt_from_classlist(l, pbtxt_path):
    pbtxt_text = ''

    for i, c in enumerate(l):
        pbtxt_text += 'item {\n    id: ' + str(
            i + 1) + '\n    display_name: "' + c + '"\n}\n\n'

    with open(pbtxt_path, "w+") as pbtxt_file:
        pbtxt_file.write(pbtxt_text)

print("Funciones auxiliares para label_map definidas")    

Funciones auxiliares para label_map definidas


In [10]:
#@title Generar label_map_pbtx

#Por las dudas, se elimina los archivos si ya existen (para asegurar su actualización)
if os.path.isfile(label_map_pbtxt_fname):
  os.remove(label_map_pbtxt_fname)
  print(label_map_pbtxt_fname, " eliminado.")

print("\n")
print("-- generando label_map_pbtxt: ", label_map_pbtxt_fname)
pbtxt_from_csv(train_csv_fname, label_map_pbtxt_fname)
print("> label_map_pbtxt generado.")

/content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/label_map.pbtxt  eliminado.


-- generando label_map_pbtxt:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/label_map.pbtxt
> label_map_pbtxt generado.


5) Generar TFRecords en base a los XMLs correspondientes a las imágenes:

In [11]:
#@title Definir funciones auxiliares para TFRecords
def generate_TFrecords(pbtxt_input, csv_input, image_dir, output_path):
    class_dict = class_dict_from_pbtxt(pbtxt_input)

    writer = tf.io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = pd.read_csv(csv_input)
    grouped = __split(examples, 'filename')

    for group in tqdm(grouped, desc='groups'):
        tf_example = create_tf_example(group, path, class_dict)
        writer.write(tf_example.SerializeToString())

    writer.close()

def create_tf_example(group, path, class_dict):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)),
                        'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        if set(['xmin_rel', 'xmax_rel', 'ymin_rel', 'ymax_rel']).issubset(
                set(row.index)):
            xmin = row['xmin_rel']
            xmax = row['xmax_rel']
            ymin = row['ymin_rel']
            ymax = row['ymax_rel']

        elif set(['xmin', 'xmax', 'ymin', 'ymax']).issubset(set(row.index)):
            xmin = row['xmin'] / width
            xmax = row['xmax'] / width
            ymin = row['ymin'] / height
            ymax = row['ymax'] / height

        xmins.append(xmin)
        xmaxs.append(xmax)
        ymins.append(ymin)
        ymaxs.append(ymax)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_dict[row['class']])

    tf_example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'image/height':
                dataset_util.int64_feature(height),
                'image/width':
                dataset_util.int64_feature(width),
                'image/filename':
                dataset_util.bytes_feature(filename),
                'image/source_id':
                dataset_util.bytes_feature(filename),
                'image/encoded':
                dataset_util.bytes_feature(encoded_jpg),
                'image/format':
                dataset_util.bytes_feature(image_format),
                'image/object/bbox/xmin':
                dataset_util.float_list_feature(xmins),
                'image/object/bbox/xmax':
                dataset_util.float_list_feature(xmaxs),
                'image/object/bbox/ymin':
                dataset_util.float_list_feature(ymins),
                'image/object/bbox/ymax':
                dataset_util.float_list_feature(ymaxs),
                'image/object/class/text':
                dataset_util.bytes_list_feature(classes_text),
                'image/object/class/label':
                dataset_util.int64_list_feature(classes),
            }))
    return tf_example

def __split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [
        data(filename, gb.get_group(x))
        for filename, x in zip(gb.groups.keys(), gb.groups)
    ]

def class_dict_from_pbtxt(pbtxt_path):
    # open file, strip \n, trim lines and keep only
    # lines beginning with id or display_name
    data = [
        l.rstrip('\n').strip()
        for l in open(pbtxt_path, 'r', encoding='utf-8-sig')
        if 'id:' in l or 'display_name:'
    ]
    ids = [int(l.replace('id:', '')) for l in data if l.startswith('id')]
    names = [
        l.replace('display_name:', '').replace('"', '').strip() for l in data
        if l.startswith('display_name')
    ]

    #print(data)

    # join ids and display_names into a single dictionary
    class_dict = {}
    for i in range(len(ids)):
        class_dict[names[i]] = ids[i]

    return class_dict


print("Funciones auxiliares para TFRecords definidas")   

Funciones auxiliares para TFRecords definidas


In [13]:
#@title Generar TFRecords

# por las dudas, se elimina los archivos si ya existen (para asegurar su actualización)
if os.path.isfile(train_record_fname):
  os.remove(train_record_fname)
  print(train_record_fname, " eliminado.")

if os.path.isfile(test_record_fname):
  os.remove(test_record_fname)
  print(test_record_fname, " eliminado.")

print("\n")
print("-- generando train_TFRecords: ", train_record_fname)
generate_TFrecords(label_map_pbtxt_fname, train_csv_fname, train_image_dir, train_record_fname)
print("\n> train_TFRecords generado.")
print("\n")
print("-- generando test_TFRecords: ", test_record_fname)
generate_TFrecords(label_map_pbtxt_fname, test_csv_fname, test_image_dir, test_record_fname)
print("\n> test_TFRecords generado.")

/content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/train.record  eliminado.
/content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/test.record  eliminado.


-- generando train_TFRecords:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/train.record


groups: 100%|██████████| 3110/3110 [00:13<00:00, 238.74it/s]



> train_TFRecords generado.


-- generando test_TFRecords:  /content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/test.record


groups: 100%|██████████| 346/346 [00:01<00:00, 243.15it/s]


> test_TFRecords generado.
